# Staff Scheduling
This is a solution to Exercise 3.4-14 from *Introduction to Operations Research*, 11th edition, by Hillier and Lieberman.

A university needs to staff a computer lab.  There are six workers, each with their own pay scale, minimum hours of work per week they are guaranteed to have, and number of hours available to work on those days.

| Workers      | Wage | Minimum Hours | Monday | Tuesday | Wednesday | Thursday | Friday
| ----------- | ----------- | --- | --- | --- | --- | --- | --- |
| KC | 25 | 8 | 6 | 0 | 6 | 0 | 6 |
| DH | 26 | 8 | 0 | 6 | 0 | 6 | 0 |
| HB | 24 | 8 | 4 | 8 | 4 | 0 | 4 |
| SC | 23 | 8 | 5 | 5 | 5 | 0 | 5 |
| KS | 28 | 7 | 3 | 0 | 3 | 8 | 0 |
| NK | 30 | 7 | 0 | 0 | 0 | 6 | 2 |

The lab needs to be staffed 14 hours a day a staff member.  The university wishes to minimize the total wages.


In [8]:
# !pip install -q pulp
import pandas as pd
import numpy as np
import pulp as pl

We build the model to be solved using `PuLP`.

In [9]:
workers = ["KC", "DH", "HB", "SC", "KS", "NK"]
wage = [25, 26, 24, 23, 28, 30]
min_hours = [8, 8, 8, 8, 7, 7]
monday = [6, 0, 4, 5, 3, 0]
tuesday = [0, 6, 8, 5, 0, 0]
wednesday = [6, 0, 4, 5, 3, 0]
thursday = [0, 6, 0, 0, 8, 6]
friday = [6, 0, 4, 5, 0, 2]

data = {'Workers': workers,
       'Wage': wage,
       'Hours': min_hours,
       'Monday' : monday,
       'Tuesday': tuesday,
       'Wednesday': wednesday,
       'Thursday': thursday,
       'Friday': friday}

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

info = pd.DataFrame(data = data)
info.set_index('Workers', inplace = True)
info

,Wage,Hours,Monday,Tuesday,Wednesday,Thursday,Friday
Workers,,,,,,,
KC,25,8,6,0,6,0,6
DH,26,8,0,6,0,6,0
HB,24,8,4,8,4,0,4
SC,23,8,5,5,5,0,5
KS,28,7,3,0,3,8,0
NK,30,7,0,0,0,6,2


We create a `schedule` variable, which consists of a value $x_{ij}$ for how many hours staffer $i$ is scheduled to work on day $j$.

In [10]:
# Use the following code to ensure the solution integer valued.
# schedule = pl.LpVariable.dicts("Schedule", (workers, days), 
#                               lowBound = 0, 
#                               cat =  pl.LpInteger) 
schedule = pl.LpVariable.dicts("Schedule", (workers, days), 
                               lowBound = 0)

Let $W$ denote the set of workers, $D$ the days to be scheduled, $c_i$ the wage for worker $i$, $h_i$ the minimum number of hours worker $i$ is scheduled to work, $A_{ij}$ the number of hours worker $i$ is available to work on day $j$.

The problem is thus: \
**Minimize:** $$Z = \sum_{i \in W} \sum_{j \in D} c_i x_{ij}$$
**Subject To:** \
Availability Contraint: $$ X \le A$$
Minimum Hours Constraint:
$$ \sum_{j \in D} x_{ij} \ge h_i, \quad i \in W$$
Fully Scheduled Constraint:
$$ \sum_{i \in W} x_{ij} = 14, \quad j \in D$$
Nonnegativity:
$$x_{ij} \ge 0$$

Note that with the equality in the Fully Scheduled Constraint, a solution may not exist (Consider if each worker is to be scheduled 14 hours per week, then some workers may need to be in the lab at the same time).  To guarantee a solution, use $>=$ instead of $=$.


In [11]:
model = pl.LpProblem("Scheduling", pl.LpMinimize)

model += pl.lpSum([schedule[worker][day] for day in days] * info.at[worker,'Wage'] for worker in workers)

for worker in workers:
    for day in days:
        model += schedule[worker][day] <= info.at[worker,day], "Availability"+worker+day
        
for worker in workers:
    model += pl.lpSum([schedule[worker][day] for day in days]) >= info.at[worker,"Hours"], "FullyEmployed"+worker

for day in days:
    model += pl.lpSum([schedule[worker][day] for worker in workers]) == 14, "FullyScheduled"+day

# model

In [12]:
print("The result of solving the model:", pl.LpStatus[model.solve()])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/stephen/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/x7/cj8wflhs4zq8nnyb67b_rgr40000gn/T/fc861a6b50014d29b256765ebe0e6845-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/x7/cj8wflhs4zq8nnyb67b_rgr40000gn/T/fc861a6b50014d29b256765ebe0e6845-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46 COLUMNS
At line 167 RHS
At line 209 BOUNDS
At line 210 ENDATA
Problem MODEL has 41 rows, 30 columns and 90 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 11 (-30) rows, 18 (-12) columns and 36 (-54) elements
0  Obj 669.79995 Primal inf 64.599993 (11)
11  Obj 1755
Optimal - objective value 1755
After Postsolve, objective 1755, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1755 - 11 iterations time 0.002, Presolve 0.00
Option for printingOp

A solution was found for the problem that is optimal, one which requires the following minimum total wages:

In [13]:
print(f"Total Wages: ${pl.value(model.objective):.2f}")

Total Wages: $1755.00


We see the final schedule for the workers in the table below.  Note that the solution may not be unique.

In [16]:
final_schedule = {}
for worker in workers:
   final_schedule[worker] = [schedule[worker][day].varValue for day in days]
fs = pd.DataFrame.from_dict(data = final_schedule, orient = 'index', columns = days)
fs

,Monday,Tuesday,Wednesday,Thursday,Friday
KC,2.0,0.0,4.0,0.0,3.0
DH,0.0,2.0,0.0,6.0,0.0
HB,4.0,7.0,4.0,0.0,4.0
SC,5.0,5.0,5.0,0.0,5.0
KS,3.0,0.0,1.0,3.0,0.0
NK,0.0,0.0,0.0,5.0,2.0
